In [16]:
import json

# Helper function to convert latitude and longitude to cartesian coordinates
def convert_to_grid_coords(lat, lng, lat_min, lat_max, lng_min, lng_max, grid_size=1000, depot_position=(500, 500)):
    # Normalizing latitude and longitude
    lat_norm = (lat - lat_min) / (lat_max - lat_min)
    lng_norm = (lng - lng_min) / (lng_max - lng_min)

    # Scaling to the grid size
    x = int(lng_norm * grid_size)
    y = int(lat_norm * grid_size)

    # Translating so that depot is at the specified position
    x += (depot_position[0] - int(grid_size / 2))
    y += (depot_position[1] - int(grid_size / 2))

    return x, y

# Path to the JSON file
file_path_json = '/Users/cstenico/Documents/shire/tcc-mba/loggibud/data/025/vrppd-instances-1.0/train/pa-0/cvrp-0-pa-0.json'


# Loading the JSON data
with open(file_path_json, 'r') as file:
    json_data = json.load(file)

# Extracting depot data
depot = json_data['depot']

partial_demands = json_data['demands'][:100]

# Finding the min and max latitude and longitude values from the JSON data
latitudes = [demand['point']['lat'] for demand in partial_demands] + [depot['lat']]
longitudes = [demand['point']['lng'] for demand in partial_demands] + [depot['lng']]
lat_min, lat_max = min(latitudes), max(latitudes)
lng_min, lng_max = min(longitudes), max(longitudes)

# Converting the depot and demands to VRP format with the new coordinate system
depot_coords = convert_to_grid_coords(depot['lat'], depot['lng'], lat_min, lat_max, lng_min, lng_max)
demands = [f"1\t0"]  # Demand at depot is zero

# Preparing nodes with the new coordinate system
nodes = [f"1\t{depot_coords[0]}\t{depot_coords[1]}"]  # Depot as the first node
for i, demand in enumerate(partial_demands, start=2):
    node_coords = convert_to_grid_coords(demand['point']['lat'], demand['point']['lng'], lat_min, lat_max, lng_min, lng_max)
    nodes.append(f"{i}\t{node_coords[0]}\t{node_coords[1]}")
    demands.append(f"{i}\t{demand['size']}")

# Constructing the VRP file content
vrp_content = f"NAME : {json_data['name']}\n"
vrp_content += "TYPE : CVRP\n"
vrp_content += f"DIMENSION : {len(nodes)}\n"
vrp_content += "EDGE_WEIGHT_TYPE : EUC_2D\n"
vrp_content += f"CAPACITY : {json_data['vehicle_capacity']}\n"
vrp_content += "NODE_COORD_SECTION\n"
vrp_content += '\n'.join(nodes)
vrp_content += "\nDEMAND_SECTION\n"
vrp_content += '\n'.join(demands)
vrp_content += "\nDEPOT_SECTION\n1\n-1\nEOF"


In [17]:
vrp_content

'NAME : cvrp-0-pa-0\nTYPE : CVRP\nDIMENSION : 101\nEDGE_WEIGHT_TYPE : EUC_2D\nCAPACITY : 180\nNODE_COORD_SECTION\n1\t913\t367\n2\t923\t439\n3\t927\t454\n4\t940\t479\n5\t935\t407\n6\t956\t465\n7\t931\t457\n8\t915\t434\n9\t945\t397\n10\t951\t477\n11\t928\t415\n12\t945\t456\n13\t923\t439\n14\t929\t475\n15\t945\t502\n16\t930\t466\n17\t927\t454\n18\t931\t445\n19\t935\t449\n20\t938\t451\n21\t921\t471\n22\t915\t472\n23\t955\t424\n24\t926\t476\n25\t922\t466\n26\t948\t461\n27\t936\t405\n28\t958\t474\n29\t946\t452\n30\t939\t428\n31\t922\t467\n32\t950\t512\n33\t926\t446\n34\t921\t473\n35\t77\t303\n36\t108\t332\n37\t95\t118\n38\t60\t336\n39\t69\t395\n40\t74\t405\n41\t91\t300\n42\t108\t302\n43\t74\t263\n44\t106\t311\n45\t96\t368\n46\t70\t240\n47\t62\t328\n48\t104\t175\n49\t86\t299\n50\t85\t325\n51\t101\t314\n52\t0\t547\n53\t92\t266\n54\t887\t406\n55\t880\t456\n56\t898\t385\n57\t874\t347\n58\t872\t433\n59\t889\t361\n60\t456\t368\n61\t462\t1000\n62\t506\t151\n63\t418\t915\n64\t476\t277\n65\t530\t174\

In [18]:
output_file_path = 'instances/PA_1/PA.vrp'

# Saving the VRP content to a file
with open(output_file_path, 'w') as file:
    file.write(vrp_content)

In [19]:
from vrp.data_utils import InstanceBlueprint

blue_print = InstanceBlueprint(nb_customers=len(partial_demands) - 1,
depot_position='R',
customer_position='R',
nb_customer_cluster=None,
demand_type='inter',
demand_min=min(demand['size'] for demand in partial_demands),
demand_max=max(demand['size'] for demand in partial_demands),
capacity=json_data['vehicle_capacity'],
grid_size=1000)

In [20]:
blue_print.__dict__


{'nb_customers': 99,
 'depot_position': 'R',
 'customer_position': 'R',
 'nb_customers_cluster': None,
 'demand_type': 'inter',
 'demand_min': 1,
 'demand_max': 10,
 'capacity': 180,
 'grid_size': 1000}